In [1]:
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

import pandas as pd

In [17]:
folder = "skin"

v_client_data = pd.read_csv(f'datasets/{folder}/data_V.csv')
v_client_data = v_client_data.to_numpy()

X_train = v_client_data

X_train = np.array(X_train).reshape(-1, 1)

W1_v = np.random.randn(5, 1)
b1 = np.random.randn(5, 1)

Z1_v = np.dot(W1_v, X_train.T) + b1

In [18]:
with open("aes_key.bin", "rb") as key_file:
    key = key_file.read()

In [21]:
# Step 1: Create a float matrix
dZ1 = Z1_v.astype(np.float32)

dZ1.shape

(5, 245057)

In [22]:
# Split dZ1 into two parts
dZ1_1 = dZ1[:, 0:10000]
dZ1_2 = dZ1[:, 10000:245057]

In [23]:
def encrypt_matrix_part(matrix_part, key):
    cipher = AES.new(key, AES.MODE_ECB)
    bytes_part = matrix_part.astype(np.float32).tobytes()
    padded = pad(bytes_part, AES.block_size)
    encrypted = cipher.encrypt(padded)
    return encrypted, matrix_part.shape  # Also return shape for reconstruction

In [24]:
# Encrypt each part independently
enc1, shape1 = encrypt_matrix_part(dZ1_1, key)
enc2, shape2 = encrypt_matrix_part(dZ1_2, key)

In [25]:
print(shape1)
print(shape2)

(5, 10000)
(5, 235057)


In [26]:
# Merge encrypted data to send to server
merged_encrypted = enc1 + enc2

In [27]:
print(len(enc1))
print(len(enc2))
print(len(merged_encrypted))

200016
4701152
4901168


In [11]:
# Split merged encrypted data using length info
enc1_received = merged_encrypted[0:20016]
enc2_received = merged_encrypted[20016:66208]

In [27]:
cipher = AES.new(key, AES.MODE_ECB)

decrypted_padded = cipher.decrypt(enc2_received)

print(len(decrypted_padded))

46192


In [28]:
decrypted = unpad(decrypted_padded, AES.block_size)
print(len(decrypted))

46180


In [29]:
Z1_v = np.frombuffer(decrypted, dtype=np.float32).reshape([5, 2208])

ValueError: cannot reshape array of size 11545 into shape (5,2208)

In [30]:
5*2208

11040

In [ ]:
def decrypt_matrix_part(encrypted_bytes, shape, key):
    
    return matrix

In [13]:
# Each H-client receives their encrypted data and shape
dZ1_part = decrypt_matrix_part(enc1_received, [5,400], key)

In [14]:
dZ2_part = decrypt_matrix_part(enc2_received, [5,972], key)

In [15]:
dZ1_2 - dZ2_part

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:


# Step 2: Convert to bytes
matrix_bytes = matrix.tobytes()

# Step 3: Pad the bytes (AES block size = 16 bytes)
padded_bytes = pad(matrix_bytes, AES.block_size)

# Step 4: Encrypt

cipher = AES.new(key, AES.MODE_ECB)
encrypted = cipher.encrypt(padded_bytes)

print(encrypted)

in_buffer = np.frombuffer(encrypted, dtype=np.uint8)

encrypted_bytes = in_buffer.tobytes()

# --- Store or transmit the `encrypted` bytes ---

# Step 5: Decrypt
decipher = AES.new(key, AES.MODE_ECB)
decrypted_padded = decipher.decrypt(encrypted_bytes)

# Step 6: Unpad and convert back to matrix
decrypted_bytes = unpad(decrypted_padded, AES.block_size)
restored_matrix = np.frombuffer(decrypted_bytes, dtype=np.float32).reshape(matrix.shape)

print("Restored Matrix:\n", restored_matrix)


Original Matrix:
 [[ 1.1173271   0.01038362  0.72780794 ...  4.024318    0.9176144
  -3.8777122 ]
 [ 6.179649   -1.2982858  -3.1410046  ...  1.1333587  -6.0132847
  -7.07616   ]
 [ 5.6536183   1.4635429  -0.862999   ... -1.949078   -3.2760894
   2.189418  ]
 [ 2.7821097   0.8804611  -1.1167172  ... -4.1418347  -2.7964952
   4.0928974 ]
 [-6.7507005  -1.9097028   0.6606696  ...  1.599122    3.3755956
  -2.3881695 ]]
b'E\x19]` \xb9\xd2\x1a\xe8\x835\xe7d8k\x00F\xb5\xcb\xd8\xb5O\x97\x8a\xc3\xe1\xa2\xa9\x1f\xbf\xd50\xd5\xe9&\x8b\nAw[\xcdi4V]\xe1\xd5\x89\x9br\xb5\xcd\xcaZ \x1aB\xd6r\xf5\x04"`\xe0\xb1\xc0\xc6\x91n\xd0\x07\x82\xc7\x04\xc5\xb9\xe7\x1c\x98p\xae\xdd\xae\xefp\n?\xa3\xf3\xdeh\xedx\xf3G\xa2\x1d\xaa\x8a\xf0nW\xa1*6\xb8\xc5+\x8d\x1c^l!\x94\xb0\t\xf2$\x06\r\x11t\xf7\xd4\xc2l\xd5\x113L\xf0K\x88\xc3\x81\x8b\'\x19\xcc\xc1\xb0\x88\xde{c\x15\xcbGy\x9d\xc9CF\xac^L\xe4\xeb9\xc2|\x10\x0b\x99\xf4\xf7m\x04\xd2\xfav\x05\x91\x80\xc5\x1d\x02\xec\x88/\x0c\xfe\xc8\x10\xd0.\xc1\t\xbb\xcf8\xd9\xbeqVG\x

In [ ]:
with open("aes_key.bin", "wb") as key_file:
    key_file.write(key)

In [1]:
import pandas as pd

In [2]:
# Load client training data from CSV file
folder = "pima"

# X_train = pd.read_csv(f'datasets/{folder}/data_H_1.csv')
# X_train = X_train.to_numpy()  # Convert to NumPy array for easier manipulation

# Read the first mask from the CSV file
mask_client_H_1 = pd.read_csv(f'masks/{folder}/mask_client_H_1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'masks/pima/mask_client_H_1.csv'

In [ ]:
Z1_v = np.dot(W1_v, X_train.T) + b1

Z1_bytes = Z1_v.tobytes()

# Step 3: Pad the bytes (AES block size = 16 bytes)
Z1_padded_bytes = pad(Z1_bytes, AES.block_size)

In [ ]:
# Step 4: Encrypt
key = get_random_bytes(16)  # AES-128
cipher = AES.new(key, AES.MODE_ECB)

encrypted_Z1 = cipher.encrypt(Z1_padded_bytes)

In [ ]:
# Step 5: Decrypt
decipher = AES.new(key, AES.MODE_ECB)
decrypted_Z1_padded = decipher.decrypt(encrypted_Z1)

In [ ]:
# Step 6: Unpad and convert back to matrix
decrypted_bytes = unpad(decrypted_Z1_padded, AES.block_size)
decrypted_Z1 = np.frombuffer(decrypted_bytes, dtype=np.float32).reshape(Z1_v.shape)

In [ ]:
print(len(ciphertext))

In [ ]:
# Decrypt
decipher = AES.new(key, AES.MODE_ECB)
decrypted_bytes = unpad(decipher.decrypt(ciphertext), AES.block_size)

In [ ]:
print(len(decrypted_bytes))

In [ ]:
# Convert back to matrix
Z1_decrypt = np.frombuffer(decrypted_bytes, dtype=np.float32).reshape(W1_v.shape)

In [ ]:
print(cipher)

In [ ]:
# Encrypt


In [ ]:
# Decrypt
decipher = AES.new(key, AES.MODE_ECB)
decrypted_bytes = unpad(decipher.decrypt(ciphertext), AES.block_size)

In [ ]:
print(recovered_matrix)
print(matrix)

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import numpy as np

In [ ]:
from Crypto.PublicKey import RSA
# === STEP 1: Generate RSA key pair ===
key = RSA.generate(2048)  # 2048-bit secure RSA key
private_key = key
public_key = key.publickey()

In [ ]:
# === STEP 2: Generate AES key ===
aes_key = get_random_bytes(16)  # AES-128

In [ ]:
# === STEP 3: Encrypt AES key using RSA public key ===
rsa_cipher = PKCS1_OAEP.new(public_key)
encrypted_aes_key = rsa_cipher.encrypt(aes_key)

In [ ]:
# === STEP 4: Decrypt AES key using RSA private key ===
rsa_decipher = PKCS1_OAEP.new(private_key)
decrypted_aes_key = rsa_decipher.decrypt(encrypted_aes_key)

In [ ]:
# === STEP 5: Confirm key is recovered correctly ===
print(f"Keys match? {aes_key == decrypted_aes_key}")  # Should print: True

In [ ]:
print(aes_key)
print(decrypted_aes_key)

In [15]:
import numpy as np

# Set the size for the masks
mask_size = (5, 1372)

# Generate three random masks with values between -5 and 5
masks = np.random.uniform(-5, 5, size=(3, *mask_size))

# Ensure the sum of the masks is zero by adjusting them
mask_sum = np.sum(masks, axis=0)
masks -= mask_sum / 3

# Verify the sum of each mask is approximately zero
sum_masks = np.sum(masks, axis=0)

# Print the generated masks and their sum to verify the condition
print("Generated Masks:")
print(masks)
print("\nSum of masks (should be near zero):")
print(sum_masks)


Generated Masks:
[[[-2.07220328 -4.20311485 -4.14264116 ...  0.28591067  0.8702518
    0.36773425]
  [-3.15987576  0.59602235 -0.8400391  ... -4.03166544 -1.43742445
    2.73618214]
  [ 0.04624472  2.18720771 -4.79621844 ...  1.06184346 -0.83965979
    1.0628118 ]
  [-0.37760208 -3.03347477  3.00820831 ...  1.1906011  -1.14953249
    1.13788204]
  [ 0.62390473 -2.28533701 -0.36130731 ... -1.61155854 -3.98160557
   -1.39809982]]

 [[-1.49402391  4.48816342 -0.32624438 ... -0.61955822  0.24616896
   -3.06676294]
  [ 6.25839219 -0.33229283 -0.84385375 ...  2.91939756  1.33646843
   -1.95157854]
  [ 0.82447674  0.35574336  2.25826567 ... -3.12051631 -0.72806328
    2.12806306]
  [-0.74549115  1.51654417 -4.49825771 ...  2.3231536   1.06756987
   -3.18593223]
  [-1.38316377  4.48353932 -3.68801462 ...  2.03359012  0.55957415
   -1.72842355]]

 [[ 3.56622719 -0.28504857  4.46888555 ...  0.33364755 -1.11642076
    2.69902869]
  [-3.09851644 -0.26372952  1.68389285 ...  1.11226787  0.10095602


In [16]:
import numpy as np
import pandas as pd

# Set the size for the masks
mask_size = (5, 1372)

# Generate three random masks with values between -5 and 5
masks = np.random.uniform(-5, 5, size=(3, *mask_size))

# Ensure the sum of the masks is zero by adjusting them
mask_sum = np.sum(masks, axis=0)
masks -= mask_sum / 3

# Assign each mask to separate variables
mask_1, mask_2, mask_3 = masks

In [17]:
print(np.sum(masks, axis=0))

[[-4.44089210e-16 -5.55111512e-16  0.00000000e+00 ...  0.00000000e+00
   4.44089210e-16  8.88178420e-16]
 [-4.44089210e-16  0.00000000e+00  0.00000000e+00 ...  4.44089210e-16
   0.00000000e+00 -1.33226763e-15]
 [-8.88178420e-16  0.00000000e+00  4.44089210e-16 ...  0.00000000e+00
   0.00000000e+00  4.44089210e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  8.88178420e-16
   8.88178420e-16 -5.55111512e-17]
 [-2.22044605e-16  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -6.66133815e-16]]


In [18]:
# Verify the sum of each mask is approximately zero
sum_mask_1 = np.sum(mask_1)
sum_mask_2 = np.sum(mask_2)
sum_mask_3 = np.sum(mask_3)

In [22]:
# Save the masks to CSV files
mask_1_df = pd.DataFrame(mask_1)
mask_2_df = pd.DataFrame(mask_2)
mask_3_df = pd.DataFrame(mask_3)

mask_1_df.to_csv('mask_client_V.csv', index=False)
mask_2_df.to_csv('mask_client_H_1.csv', index=False)
mask_3_df.to_csv('mask_client_H_2.csv', index=False)

In [34]:
nx = np.array([2, 5, 5, 1])

In [35]:
 # Initialize weights and biases for each layer
W1_h = np.zeros((nx[1], nx[0]), dtype=np.float32)

W2 = np.zeros((nx[2], nx[1]), dtype=np.float32)
b2 = np.zeros((nx[2], 1), dtype=np.float32)

W3 = np.zeros((nx[3], nx[2]), dtype=np.float32)
b3 = np.zeros((nx[3], 1), dtype=np.float32)

In [36]:
def reshape_to_5_rows(matrix):
    flat = matrix.flatten()
    needed_cols = int(np.ceil(len(flat) / 5))
    padded = np.pad(flat, (0, needed_cols * 5 - len(flat)))
    reshaped = padded.reshape(5, -1)
    return reshaped
                
# Reshape and concatenate
W1_h_r = reshape_to_5_rows(W1_h)
W2_r = reshape_to_5_rows(W2)
b2_r = reshape_to_5_rows(b2)
W3_r = reshape_to_5_rows(W3)
b3_r = reshape_to_5_rows(b3)

print(W1_h_r)
print(W2_r)
print(b2_r)
print(W3_r)
print(b3_r)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [39]:
b2.shape

(5, 1)